In [1]:
import queue
from experiment_information import *
from experiments.sweep_generalization_error import *
from _version import __version__
from data import *
import pandas as np
import numpy as np
import logging
%load_ext autoreload
%autoreload 2

In [3]:
# Create a SweepExperiment
experiment = get_default_experiment()
# use json dump to save the experiment parameters
with open("sweep_experiment.json","w") as f:
    # use the NumpyEncoder to encode numpy arrays
    json.dump(experiment.__dict__,f,cls=NumpyEncoder)

In [19]:
df_experiments = None
df_state_evolution = None
df_erm = None
logger = logging.getLogger()
with DatabaseHandler(logger,"experiments/experiments.db") as dbHandler:
    df_experiments = dbHandler.get_experiments()
    df_state_evolution = dbHandler.get_state_evolutions()
    df_erm = dbHandler.get_erms()


In [20]:
# get the completed experiments with the current code version ordered by date
df_experiments = df_experiments[(df_experiments["completed"]==True) & (df_experiments["code_version"]==__version__)]
df_experiments = df_experiments.sort_values(by="date",ascending=False)
df_experiments.head(2)

,experiment_id,experiment_name,duration,code_version,date,state_evolution_repetitions,erm_repetitions,alphas,epsilons,lambdas,tau,d,erm_methods,completed
4,f2da5b3f-e115-42bb-b0c5-dfd8245f1b3c,Default Experiment,168.912740,10,2023-03-30 09:53:55,10,10,"[0.2, 1.0, 2.0, 5.0]","[0.0, 0.01, 0.2, 1.0]",[1],2.0,1000,"[""gd""]",1
3,c8ff30c5-3fa9-4161-992b-a6212bfe5330,Default Experiment,19.043362,10,2023-03-30 09:34:41,5,5,"[0.2, 1.0, 2.0, 5.0]",[0],[1],2.0,1000,"[""gd""]",1


In [21]:
# extract and print the top experiment_id
experiment_id = df_experiments.iloc[0]["experiment_id"]
print(experiment_id)

f2da5b3f-e115-42bb-b0c5-dfd8245f1b3c


In [22]:
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == experiment_id]
erm = df_erm[df_erm["experiment_id" ] == experiment_id]
gd = erm[erm["chosen_minimizer"] == "gd"]
# set alpha, epsilon, tau, lam as index in gd and state_evolution
gd = gd.set_index(["alpha","epsilon","tau","lam"])
state_evolution = state_evolution.set_index(["alpha","epsilon","tau","lam"])
# from state_evolution keep generalization_error, sigma, q, m, cosb
state_evolution = state_evolution[["generalization_error","sigma","q","m","cosb"]]
# add a suffix _state_evolution to the columns
state_evolution.columns = [col+"_state_evolution" for col in state_evolution.columns]
# from gd keep generalization_error_erm	generalization_error_overlap Q	rho	m	cosb
gd = gd[["generalization_error_erm","generalization_error_overlap","Q","rho","m","cosb"]]
# add a suffix _erm to the columns
gd.columns = [col+"_erm" for col in gd.columns]
# join the two dataframes
df = state_evolution.join(gd)
# reorder the columns
df = df[['generalization_error_state_evolution', 'generalization_error_erm_erm', 'generalization_error_overlap_erm', 'sigma_state_evolution', 'q_state_evolution','Q_erm','m_state_evolution','m_erm','cosb_state_evolution','cosb_erm', 'rho_erm']]
# average all quantities over the index and obtain also the standard deviation
df = df.groupby(level=[0,1,2,3]).agg([np.mean,np.std])
# display all columns
pd.set_option('display.max_columns', None)
df

generalization_error_state_evolution       \
                                                      mean  std   
alpha epsilon tau lam                                             
0.2   0.00    2.0 1.0                             0.496378  0.0   
      0.01    2.0 1.0                             0.496359  0.0   
      0.20    2.0 1.0                             0.495987  0.0   
      1.00    2.0 1.0                             0.494207  0.0   
1.0   0.00    2.0 1.0                             0.483627  0.0   
      0.01    2.0 1.0                             0.483519  0.0   
      0.20    2.0 1.0                             0.481337  0.0   
      1.00    2.0 1.0                             0.470981  0.0   
2.0   0.00    2.0 1.0                             0.470759  0.0   
      0.01    2.0 1.0                             0.470531  0.0   
      0.20    2.0 1.0                             0.466040  0.0   
      1.00    2.0 1.0                                  NaN  NaN   
5.0   0.00    2.0 1.0                             0.444482  0.0   
      0.01    2.0 1.0                             0.443964  0.0   
      0.20    2.0 1.0                             0.435129  0.0   
      1.00    2.0 1.0                                  NaN  NaN   

                      generalization_error_erm_erm            \
                                              mean       std   
alpha epsilon tau lam                                          
0.2   0.00    2.0 1.0                     0.479018  0.003175   
      0.01    2.0 1.0                     0.479018  0.002247   
      0.20    2.0 1.0                     0.487321  0.018159   
      1.00    2.0 1.0                     0.484513  0.016797   
1.0   0.00    2.0 1.0                     0.451602  0.004728   
      0.01    2.0 1.0                     0.450889  0.003586   
      0.20    2.0 1.0                     0.477517  0.037992   
      1.00    2.0 1.0                     0.486029  0.036434   
2.0   0.00    2.0 1.0                     0.433808  0.004460   
      0.01    2.0 1.0                     0.433958  0.004119   
      0.20    2.0 1.0                     0.501643  0.053666   
      1.00    2.0 1.0                     0.459039  0.061595   
5.0   0.00    2.0 1.0                     0.409538  0.002281   
      0.01    2.0 1.0                     0.407758  0.003770   
      0.20    2.0 1.0                     0.463456  0.107413   
      1.00    2.0 1.0                     0.417108  0.051443   

                      generalization_error_overlap_erm            \
                                                  mean       std   
alpha epsilon tau lam                                              
0.2   0.00    2.0 1.0                         0.478351  0.003343   
      0.01    2.0 1.0                         0.478622  0.002799   
      0.20    2.0 1.0                         0.487480  0.018146   
      1.00    2.0 1.0                         0.485402  0.017239   
1.0   0.00    2.0 1.0                         0.451345  0.004826   
      0.01    2.0 1.0                         0.450705  0.002339   
      0.20    2.0 1.0                         0.478838  0.039560   
      1.00    2.0 1.0                         0.491851  0.039485   
2.0   0.00    2.0 1.0                         0.433870  0.004277   
      0.01    2.0 1.0                         0.434230  0.004182   
      0.20    2.0 1.0                         0.508918  0.057298   
      1.00    2.0 1.0                         0.481704  0.043074   
5.0   0.00    2.0 1.0                         0.409330  0.002182   
      0.01    2.0 1.0                         0.407735  0.002880   
      0.20    2.0 1.0                         0.488288  0.083524   
      1.00    2.0 1.0                         0.470280  0.058486   

                      sigma_state_evolution      q_state_evolution       \
                                       mean  std              mean  std   
alpha epsilon tau lam                                                     
0.2   0.00   

In [15]:
df.columns

Index(['generalization_error_state_evolution', 'sigma_state_evolution',
       'q_state_evolution', 'm_state_evolution', 'cosb_state_evolution',
       'generalization_error_erm_erm', 'generalization_error_overlap_erm',
       'Q_erm', 'rho_erm', 'm_erm', 'cosb_erm'],
      dtype='object')

In [7]:
erm = df_erm[df_erm["experiment_id" ] == experiment_id]
erm[erm["chosen_minimizer"] == "logistic"]

,id,duration,code_version,experiment_id,Q,rho,m,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,date,chosen_minimizer,training_error,d,tau,alpha,test_loss


In [8]:
erm[erm["chosen_minimizer"] == "gd"]

,id,duration,code_version,experiment_id,Q,rho,m,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,date,chosen_minimizer,training_error,d,tau,alpha,test_loss
60,0d926e78-685b-402a-9468-799921b8f133,2.993294,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.031137,0.984257,0.024210,0.138291,0.0,1.0,0.47930,0.480426,2023-03-30 09:34:47,gd,0.0000,1000,2.0,0.2,0.692732
61,234eaf13-14c7-4388-957f-bb512fce65cf,2.893143,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.035293,0.989232,0.032515,0.174014,0.0,1.0,0.47671,0.475311,2023-03-30 09:34:47,gd,0.0000,1000,2.0,0.2,0.691818
62,74b3be64-242c-4a5a-a87f-0095eab023ae,2.962294,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.028977,1.004127,0.019844,0.116335,0.0,1.0,0.48357,0.483405,2023-03-30 09:34:47,gd,0.0000,1000,2.0,0.2,0.693013
63,06cd2682-cca3-4b46-92a5-96f3645f8941,4.230864,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.029539,0.974448,0.021160,0.124722,0.0,1.0,0.48209,0.482420,2023-03-30 09:34:48,gd,0.0000,1000,2.0,0.2,0.692771
64,6ab9d0ad-1a35-42f6-b4ac-870432e4bfea,4.197621,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.029264,1.003112,0.018021,0.105180,0.0,1.0,0.48668,0.485003,2023-03-30 09:34:48,gd,0.0000,1000,2.0,0.2,0.693719
65,fc437d24-5e25-4f79-96b5-08ed2c0aa933,3.592421,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.130657,1.065233,0.134564,0.360696,0.0,1.0,0.44735,0.447105,2023-03-30 09:34:50,gd,0.0910,1000,2.0,1.0,0.685610
66,2f72884f-0bae-4dfd-aa5e-c82bf17f6859,3.368609,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.129342,0.997018,0.125498,0.349474,0.0,1.0,0.45036,0.450107,2023-03-30 09:34:52,gd,0.1160,1000,2.0,1.0,0.686190
67,be7098a3-7edb-46e6-b0e0-940cd2f68ce4,3.494028,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.129774,1.049670,0.109981,0.297988,0.0,1.0,0.45694,0.456620,2023-03-30 09:34:52,gd,0.1170,1000,2.0,1.0,0.689596
68,46cbc35f-1f9f-439a-9358-c3953613323f,3.583394,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.134801,1.005402,0.126787,0.344397,0.0,1.0,0.45128,0.450671,2023-03-30 09:34:52,gd,0.1040,1000,2.0,1.0,0.686985
69,c685d47c-3d12-43a6-bb5a-f77e930eca37,3.486734,10,c8ff30c5-3fa9-4161-992b-a6212bfe5330,0.135410,1.093364,0.139380,0.362236,0.0,1.0,0.44548,0.446324,2023-03-30 09:34:52,gd,0.1010,1000,2.0,1.0,0.684759


In [52]:
erm[erm["chosen_minimizer"] == "L-BFGS-B"]

,id,duration,code_version,experiment_id,Q,rho,m,cosb,epsilon,lam,generalization_error_erm,generalization_error_overlap,date,chosen_minimizer,training_error,d,tau,alpha,test_loss
357,1474ae37-0e4e-42e1-819a-41a8fb93f593,4.456638,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.032707,0.983764,0.059604,0.332281,0.0,1.0,0.39217,0.392182,2023-03-25 12:43:21,L-BFGS-B,0.0000,1000,0.0,0.2,0.673257
358,09bb47e3-2f08-4165-b5b5-94c1e960538f,4.298436,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.031599,1.012520,0.058697,0.328157,0.0,1.0,0.39077,0.393572,2023-03-25 12:43:21,L-BFGS-B,0.0000,1000,0.0,0.2,0.673326
361,d4a1dbab-183a-433e-9a67-586e1a54daef,4.299042,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.031393,0.961438,0.062612,0.360392,0.0,1.0,0.38185,0.382643,2023-03-25 12:43:22,L-BFGS-B,0.0000,1000,0.0,0.2,0.671693
364,a4be7489-ae33-4a3f-a324-79352d44d229,2.776170,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.032398,0.978171,0.059626,0.334943,0.0,1.0,0.39144,0.391283,2023-03-25 12:43:26,L-BFGS-B,0.0000,1000,0.0,0.2,0.672773
370,5112b39b-e90f-4fdc-9d4f-2c049790aeb8,3.250064,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.033834,1.007658,0.067642,0.366337,0.0,1.0,0.37878,0.380612,2023-03-25 12:43:27,L-BFGS-B,0.0000,1000,0.0,0.2,0.670076
372,5a0be9d7-78d2-4335-b9db-6664674d069a,2.927937,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.179398,1.066074,0.280501,0.641406,0.0,1.0,0.27734,0.278351,2023-03-25 12:43:31,L-BFGS-B,0.0400,1000,0.0,1.0,0.605740
374,3d09ea61-e3fd-4274-be91-4e278bfc9c71,2.987275,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.178651,1.048371,0.285812,0.660420,0.0,1.0,0.26878,0.270378,2023-03-25 12:43:31,L-BFGS-B,0.0300,1000,0.0,1.0,0.603767
376,660431b9-704c-45b9-a8cf-14caa9caef39,3.097832,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.174103,1.040260,0.282984,0.664949,0.0,1.0,0.26457,0.268453,2023-03-25 12:43:31,L-BFGS-B,0.0420,1000,0.0,1.0,0.602674
383,6fbf7467-5e55-4486-bd98-2c886563d4e0,2.969160,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.178747,0.925289,0.264650,0.650749,0.0,1.0,0.27349,0.274455,2023-03-25 12:43:36,L-BFGS-B,0.0300,1000,0.0,1.0,0.605681
384,3542fb10-c4a5-435f-a60b-c095ad92a8d1,3.046048,7,e771baf8-dd50-467e-9262-2346ca9f0a0d,0.182953,1.014368,0.286957,0.666115,0.0,1.0,0.26738,0.267956,2023-03-25 12:43:36,L-BFGS-B,0.0320,1000,0.0,1.0,0.601711


In [48]:
# merge df_erm and df_state_evolution on experiment id efd6ee50-96f6-4ddd-9a84-ce6c74469637
erm = df_erm[df_erm["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
state_evolution = df_state_evolution[df_state_evolution["experiment_id" ] == "efd6ee50-96f6-4ddd-9a84-ce6c74469637"]
# print the number of rows in each dataframe
print("erm: %d, state_evolution: %d" % (erm.shape[0],state_evolution.shape[0]))
# round alpha and epsilon to 2 decimals
erm["alpha"] = erm["alpha"].round(2)
erm["epsilon"] = erm["epsilon"].round(2)
state_evolution["alpha"] = state_evolution["alpha"].round(2)
state_evolution["epsilon"] = state_evolution["epsilon"].round(2)
# keep alpha, epsilon and the generalization errors
erm = erm[["alpha","epsilon","generalization_error_erm","generalization_error_overlap"]]
state_evolution = state_evolution[["alpha","epsilon","generalization_error"]]

# join the dataframes on alpha an epsilon and add suffixes to the columns
df = erm.merge(state_evolution, on=["alpha","epsilon"], suffixes=("_erm","_state_evolution"))
# print the number of rows in the joined dataframe
print("joined: %d" % df.shape[0])
df

erm: 6, state_evolution: 6
joined: 6


/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["alpha"] = state_evolution["alpha"].round(2)
/var/folders/4p/9b_m1n3130xfbjd06qfyyqgc0000gn/T/ipykernel_6324/4173925508.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  state_evolution["epsilon"] = state_evolution["epsilon"].round(2)


,alpha,epsilon,generalization_error_erm,generalization_error_overlap,generalization_error
0,0.64,0.02,0.46955,0.440456,0.122996
1,0.64,0.00,0.46935,0.416841,0.123619
2,2.28,0.02,0.44400,0.351962,0.066441
3,5.00,0.00,0.41380,0.300245,0.045324
4,5.00,0.02,0.41235,0.287456,0.045062
5,2.28,0.00,0.44595,0.386968,0.066818
